# Debugging the scripts

### Important paths in parameter file for maker

In [ ]:
###	used by filter_makerRNAs.R
MINEXONS="2"
MAXEAED="0.9"
MAXAED="0.6"
###	used by maker
MASKED="/home/data/pest_genomics/Aphis_gossypii/annotation/pipeline/1_RNAseq/assembly.fa"
PARALLEL="5"
###

## Chronology

RUN_MAKER.SH --> Chrom1.gff --> makerRNA.tab --> RSCRIPT --> filter_makerRNA.ext

	2. Running longest contigs through maker in batches 
	3a. Extracting gene models from longest contigs 
	3b. Filtering best gene models from longest contigs using R script
	3b. Filtering best gene models from longest contigs using R script
	3c. Extracting gff of best gene models for gene training
	4a. Investigating training set for overlapping genes 

## Ask David the following

#### Rscript

Is the purpose of the Rscript to select the mRNA with matching parameters to the MINEXONS, MaxAED and MaxeAED as set in parameters.txt?

If so why do we specify a for loop of MaxAED up to 0.7 when we have set it to 0.6?

Why do we run the Rscript 3 times? First on longest contigs, then on  each MAXAED tab file and finally again in check_ol.sh?

Where did "MakerRNA.filtered.gff" came from? It showed up in check_ol.sh but it wasn't even made in run_maker, the previous script or anywhere else.

# Understanding by code blocks

##### run_maker.sh

In [ ]:
#	3a. Extracting gene models from chromosome 1 (largest contig) 
##################################################################

#From maker output for chromosome 1 contig, extract mRNA names 
#and other maker stats into makerRNAs.tab.

#for gff file: delete fasta records, select mRNA lines, columnise by ; and |,
#select cols for ID, AED, eAED, num_exons, and select only first transcripts 

##################################################################
echo "Extracting best gene models from chromosome 1 contig."
echo "Overwriting makerRNAs.tab"
rm -f makerRNAs.tab
echo $(date)


echo "Extracting gene ids from maker output for chromosome 1"
GFF="./chrom1.maker.output/chrom1_datastore/9D/45/Chromosome_1/Chromosome_1.gff"

sed '/^##FASTA$/,$d' $GFF | grep -P 'mRNA\t' | tr ';' '\t' | tr '|' '\t' |\
	cut -f9,12,13,20 | sed -e 's/^ID=//g' -e 's/_AED=//g' -e 's/_eAED=//g' |\
	grep 'mRNA-1' >> makerRNAs.tab
done
#	Finally add header
sed -i '1iID\tAED\teAED\tnum_exons' makerRNAs.tab
echo $(date)


In [ ]:

#	3b. Filtering best gene models from longest contigs using R script
##################################################################

###https://www.nature.com/articles/nrg3174.pdf
###http://gmod.org/wiki/MAKER_Tutorial#MAKER.27s_Output
###http://www.yandell-lab.org/publications/pdf/maker_current_protocols.pdf
###https://groups.google.com/forum/#!msg/maker-devel/wtmNRtRa-ko/iC4KTuIitGEJ

##################################################################

MINEXONS="2"
MAXEAED="0.7"
MAXAED="0.6"

echo "Setting threshholds for best models"
echo "Minimum number of exons: "$MINEXONS
echo "Maximum eAED score: "$MAXEAED
echo "Filtering gene models into makerRNAs.filtered.MAXAED.tab .."

for MAXAED in 0.1 0.2 0.3 0.4 0.5 0.6 0.7
do
	echo ".. with Maximum AED score:"$MAXAED
	PROG="/home/data/pest_genomics/scripts/dh/filter_makerRNAs.R"
	Rscript $PROG makerRNAs.tab $MAXAED $MAXEAED $MINEXONS
	mv makerRNAs.filtered.tab makerRNAs.filtered.${MAXAED}.tab
done
date +"%d-%b-%y %T"

##### I am confused as to why we are outputting a makerRNAs.filtered.tab for every single MaxAED?

##### run_check_ol.sh

In [ ]:
#	3b. Filtering best gene models from chromosome 1 contig using R script

##################################################################

###https://www.nature.com/articles/nrg3174.pdf
###http://gmod.org/wiki/MAKER_Tutorial#MAKER.27s_Output
###http://www.yandell-lab.org/publications/pdf/maker_current_protocols.pdf
###https://groups.google.com/forum/#!msg/maker-devel/wtmNRtRa-ko/iC4KTuIitGEJ

###################################################################

MINEXONS="2"
MAXEAED="0.7"
MAXAED="0.6"

echo "Setting threshholds for best models"
echo "Filtering gene models into makerRNAs.filtered.tab .."
echo ".. with Minimum number of exons: "$MINEXONS
echo ".. with Maximum eAED score: "$MAXEAED
echo ".. with Maximum AED score: "$MAXAED

PROG="/home/data/pest_genomics/scripts/dh/filter_makerRNAs.R"
Rscript $PROG makerRNAs.tab $MAXAED $MAXEAED $MINEXONS
echo $(date)

In [ ]:
#3c. Extracting gff of best gene models for gene training

##################################################################

#	For each selected contig, extract gff for filtered.makerRNAs
#	into makerRNAs.filtered.gff

#################################################################

echo "Extracting gff of best gene models"
cut -f1 makerRNAs.filtered.tab | sed '1d' > makerRNAs.filtered.ids
echo $(date)

echo "Extracting gff for best genemodels"
GFF="./chrom1.maker.output/chrom1_datastore/9D/45/Chromosome_1/Chromosome_1.gff"
grep -f makerRNAs.filtered.ids $GFF >> makerRNAs.filtered.gff
echo $(date)

In [ ]:
#	4a. Investigating training set for overlapping genes 
##################################################################

#	Augustus requires that
#	1.	the training set genes must not overlap each other
#	2.	the training set genes should be non-redundant
#	3.	only one transcript per gene [but we selected only the first transcript (mRNA-1) at step 3a above]

##################################################################

echo "Finding overlaps in training geneset"
bedtools sort -i makerRNAs.filtered.gff > makerRNAs.filtered.sorted.gff
grep -P 'maker\tmRNA' makerRNAs.filtered.sorted.gff | awk '{sub(/mRNA/,"mRNA_"++i)}1' - > makerRNAs.filtered.sorted.mRNA.gff
bedtools merge -n -nms -i makerRNAs.filtered.sorted.mRNA.gff | grep ';' | tr ';' '\t' | awk '{print $4"\n"$5}' - >overlapping_genes

if [ ! -s overlapping_genes ]
then
	echo "No overlapping genes found"
	touch overlapping_genes.delete 
	date +"%d-%b-%y %T"
else
	grep -w -f overlapping_genes makerRNAs.filtered.sorted.mRNA.gff >overlapping_genes.gff
	echo "Overlapping genes in training geneset are:"
	cat overlapping_genes.gff
	date +"%d-%b-%y %T"


echo "The following code suggests which genes you might want to remove from training set!!"
	COUNT=$(cat overlapping_genes|wc -l) 
	##echo $COUNT
	NUM_OLAPS=$(echo $COUNT / 2 |bc)
	##echo $NUM_OLAPS
	seq 1 $NUM_OLAPS > temp1.file
	paste temp1.file temp1.file | tr '\t' '\n' > temp2.file
	paste temp2.file overlapping_genes.gff | cut -f1,4-6,10 |awk '{$10=$4-$3}1' | awk '{print $1"\t"$2"\t"$3"\t"$4"\t"$6"\t"$5 }'|\
		sed -r 's/;.*$//g' | sort -n -k1,1 -k5,5 >overlapping_pairs
	rm temp?.file
	echo "You might want to delete the shorter gene in each of the following pairs:"
	cat overlapping_pairs
	awk 'NR%2!=0' overlapping_pairs |cut -f6 >overlapping_genes.delete
	echo "which I think are the following:"
	cat overlapping_genes.delete
	echo $(date)
fi


In [ ]:
#	4a. Removing overlapping genes from taining set 
##################################################################
if [ ! -s $DELETE ]
then
	echo "No overlapping genes to remove"
	cp best_genes/makerRNAs.filtered.gff best_genes/makerRNAs.filtered.nolaps.gff
	date +"%d-%b-%y %T"
else
	echo "Removing overlapping genes from training set"
	grep -w -v -f $DELETE best_genes/makerRNAs.filtered.gff > best_genes/makerRNAs.filtered.nolaps.gff  
	date +"%d-%b-%y %T"
fi

echo "Tidying up"
mv overlapping* best_genes
date +"%d-%b-%y %T"

# The full script that seemed to have worked

In [ ]:
#!/bin/bash

set -eu
cd 2_training
echo $(date)

#	3a. Extracting gene models from chromosome 1 (largest contig) 
##################################################################

#From maker output for chromosome 1 contig, extract mRNA names 
#and other maker stats into makerRNAs.tab.

#for gff file: delete fasta records, select mRNA lines, columnise by ; and |,
#select cols for ID, AED, eAED, num_exons, and select only first transcripts 

##################################################################
echo "Extracting best gene models from chromosome 1 contig."
echo "Overwriting makerRNAs.tab"
rm -f makerRNAs.tab
echo $(date)


echo "Extracting gene ids from maker output for chromosome 1"
GFF="./chrom1.maker.output/chrom1_datastore/9D/45/Chromosome_1/Chromosome_1.gff"

sed '/^##FASTA$/,$d' $GFF | grep -P 'mRNA\t' | tr ';' '\t' | tr '|' '\t' |\
	cut -f9,12,13,20 | sed -e 's/^ID=//g' -e 's/_AED=//g' -e 's/_eAED=//g' |\
	grep 'mRNA-1' >> makerRNAs.tab

#	Finally add header
sed -i '1iID\tAED\teAED\tnum_exons' makerRNAs.tab
echo $(date)

#	3b. Filtering best gene models from longest contigs using R script

##################################################################

###https://www.nature.com/articles/nrg3174.pdf
###http://gmod.org/wiki/MAKER_Tutorial#MAKER.27s_Output
###http://www.yandell-lab.org/publications/pdf/maker_current_protocols.pdf
###https://groups.google.com/forum/#!msg/maker-devel/wtmNRtRa-ko/iC4KTuIitGEJ

###################################################################

MINEXONS="2"
MAXEAED="0.9"
MAXAED="0.6"

echo "Setting threshholds for best models"
echo "Filtering gene models into makerRNAs.filtered.tab .."
echo ".. with Minimum number of exons: "$MINEXONS
echo ".. with Maximum eAED score: "$MAXEAED
echo ".. with Maximum AED score: "$MAXAED

PROG="/home/data/pest_genomics/scripts/dh/filter_makerRNAs.R"
Rscript $PROG makerRNAs.tab $MAXAED $MAXEAED $MINEXONS
echo $(date)

#	3b. Filtering best gene models from chromosome 1 contig using R script

##################################################################

###https://www.nature.com/articles/nrg3174.pdf
###http://gmod.org/wiki/MAKER_Tutorial#MAKER.27s_Output
###http://www.yandell-lab.org/publications/pdf/maker_current_protocols.pdf
###https://groups.google.com/forum/#!msg/maker-devel/wtmNRtRa-ko/iC4KTuIitGEJ

###################################################################

MINEXONS="2"
MAXEAED="0.7"
MAXAED="0.6"

echo "Setting threshholds for best models"
echo "Filtering gene models into makerRNAs.filtered.tab .."
echo ".. with Minimum number of exons: "$MINEXONS
echo ".. with Maximum eAED score: "$MAXEAED
echo ".. with Maximum AED score: "$MAXAED

PROG="/home/data/pest_genomics/scripts/dh/filter_makerRNAs.R"
Rscript $PROG makerRNAs.tab $MAXAED $MAXEAED $MINEXONS
echo $(date)

#3c. Extracting gff of best gene models for gene training

##################################################################

#	For each selected contig, extract gff for filtered.makerRNAs
#	into makerRNAs.filtered.gff

#################################################################

echo "Extracting gff of best gene models"
cut -f1 makerRNAs.filtered.tab | sed '1d' > makerRNAs.filtered.ids
echo $(date)

echo "Extracting gff for best genemodels"
GFF="./chrom1.maker.output/chrom1_datastore/9D/45/Chromosome_1/Chromosome_1.gff"
grep -f makerRNAs.filtered.ids $GFF >> makerRNAs.filtered.gff
echo $(date)

#	4a. Investigating training set for overlapping genes 
##################################################################

#	Augustus requires that
#	1.	the training set genes must not overlap each other
#	2.	the training set genes should be non-redundant
#	3.	only one transcript per gene [but we selected only the first transcript (mRNA-1) at step 3a above]

##################################################################

echo "Finding overlaps in training geneset"
bedtools sort -i makerRNAs.filtered.gff > makerRNAs.filtered.sorted.gff
grep -P 'maker\tmRNA' makerRNAs.filtered.sorted.gff | awk '{sub(/mRNA/,"mRNA_"++i)}1' - > makerRNAs.filtered.sorted.mRNA.gff
bedtools merge -n -nms -i makerRNAs.filtered.sorted.mRNA.gff | grep ';' | tr ';' '\t' | awk '{print $4"\n"$5}' - >overlapping_genes

if [ ! -s overlapping_genes ]
then
	echo "No overlapping genes found"
	touch overlapping_genes.delete 
	date +"%d-%b-%y %T"
else
	grep -w -f overlapping_genes makerRNAs.filtered.sorted.mRNA.gff >overlapping_genes.gff
	echo "Overlapping genes in training geneset are:"
	cat overlapping_genes.gff
	date +"%d-%b-%y %T"


echo "The following code suggests which genes you might want to remove from training set!!"
	COUNT=$(cat overlapping_genes|wc -l) 
	##echo $COUNT
	NUM_OLAPS=$(echo $COUNT / 2 |bc)
	##echo $NUM_OLAPS
	seq 1 $NUM_OLAPS > temp1.file
	paste temp1.file temp1.file | tr '\t' '\n' > temp2.file
	paste temp2.file overlapping_genes.gff | cut -f1,4-6,10 |awk '{$10=$4-$3}1' | awk '{print $1"\t"$2"\t"$3"\t"$4"\t"$6"\t"$5 }'|\
		sed -r 's/;.*$//g' | sort -n -k1,1 -k5,5 >overlapping_pairs
	rm temp?.file
	echo "You might want to delete the shorter gene in each of the following pairs:"
	cat overlapping_pairs
	awk 'NR%2!=0' overlapping_pairs |cut -f6 >overlapping_genes.delete
	echo "which I think are the following:"
	cat overlapping_genes.delete
	echo $(date)
fi


echo "Tidying up"
mkdir "best_gene_models"
mv makerRNAs* ./best_gene_models

echo "Proceed to next script"
echo $(date)
exit

# Ask Rob the following

 -  Given that most fungal genes overlap, as is the case in which we found 57 genes in chromosome 1 overlapping. Should we skip the step deleting the overlaps.
 - Are the following steps still relevant for run_check_nr? Investigating training set for non-redundant proteins and conversion of gff and fasta nto genbank format.
 - When running blastn against RepeatMasker, is the query unaligned.fa or consesi.fa? What database should we use?

# Answers

 - There are only 57 overlapping genes but thousands in chromosome 1. Therefore delete overlapping genes.
 - Convert to genbank format and remove overlapping genes.
 - Use Decypher for blastn on consensi.fa
 
 ### Why do we convert the gff and genomic assembly into genbank format?